# LearningPySpark_Ch03
## Chapter 3: DataFrames
This notebook contains sample code from Chapter 4 of <a href="https://render.githubusercontent.com/view/ipynb?commit=edcaf5ed42558fb08759f70a01201e00aa2d49f7&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f5061636b745075626c697368696e672f4c6561726e696e672d5079537061726b2f656463616635656434323535386662303837353966373061303132303165303061613264343966372f4368617074657230322f4c6561726e696e675079537061726b5f4368617074657230332e6970796e62&nwo=PacktPublishing%2FLearning-PySpark&path=Chapter02%2FLearningPySpark_Chapter03.ipynb&repository_id=83264169">Learning PySpark</a> focusing on PySpark and DataFrames.

 - Whenever PySpark execute some code with RDDs, PySpark Driver, the $Spark Context$ uses $Py4j$ 'JVM' using the JavaSparkContext. 
 - Any RDD trasnformations are mapped to PythonRDD objects in java
 - these tasks are pushed out to the Spark Worker(s), PythonRDD objects launch Python subprocesses using pipes to send both code and data to be processed within Python:

![img1](img/1.jpg)

### Generate your own DataFrame
- Instead of accessing the file system, let's create a DataFrame by generating the data. 
- In this case, we'll first create the stringRDD RDD and then convert it into a DataFrame when we're reading stringJSONRDD using spark.read.json.

In [1]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)
stringJSONRDD

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475

In [2]:
stringJSONRDD.collect()

[' \n  { "id": "123",\n    "name": "Katie",\n    "age": 19,\n    "eyeColor": "brown"\n  }',
 '{\n    "id": "234",\n    "name": "Michael",\n    "age": 22,\n    "eyeColor": "green"\n  }',
 '{\n    "id": "345",\n    "name": "Simone",\n    "age": 23,\n    "eyeColor": "blue"\n  }']

In [3]:
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD)
swimmersJSON.collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [4]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [5]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [6]:
# SQL Query
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [7]:
#%sql 
#-- Query Data
#select * from swimmersJSON

#### Inferring the Schema Using Reflection
Note that Apache Spark is inferring the schema using reflection; <br>
i.e. it automaticlaly determines the schema of the data based on reviewing the JSON data.

In [8]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



 - Notice that Spark was able to determine infer the schema (when reviewing the schema using .printSchema).
 - But what if we want to programmatically specify the schema?

#### Programmatically Specifying the Schema
In this case, let's specify the schema for a CSV text file.

In [9]:
from pyspark.sql.types import *

# Generate our own CSV data 
#   This way we don't have to access the file system yet.
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])
stringCSVRDD.collect()

[(123, 'Katie', 19, 'brown'),
 (234, 'Michael', 22, 'green'),
 (345, 'Simone', 23, 'blue')]

In [10]:
# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [11]:
# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [12]:
# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [13]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [14]:
%sql 
#-- Query the data
select * from swimmers

SyntaxError: invalid syntax (<ipython-input-14-76420af740bd>, line 3)

In [15]:
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



As you can see from above, we can programmatically apply the schema instead of allowing the Spark engine to infer the schema via reflection.

Additional Resources include:
 - <a href="https://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html">PySpark API Reference</a>
 - <a href="https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema">Spark SQL, DataFrames, and Datasets Guide </a>: This is in reference to Programmatically Specifying the Schema using a CSV file.

|| SparkSession

Notice that we're no longer using sqlContext.read... but instead spark.read.... This is because as part of Spark 2.0, HiveContext,  SQLContext, StreamingContext, SparkContext have been merged together into the Spark Session spark.
 - Entry point for reading data
 - Working with metadata
 - Configuration
 - Cluster resource management

For more information, please refer to How to use <a href="http://bit.ly/2br0Fr1">SparkSession</a> in Apache Spark 2.0 (http://bit.ly/2br0Fr1).

### Querying with SQL

With DataFrames, you can start writing your queries using Spark SQL - a SQL dialect that is compatible with the Hive Query Language (or HiveQL).

In [16]:
# SQL문 실행 / 데이터 출력 
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



 - get the row count

In [17]:
spark.sql("select count(*) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



 Note, %sql로 interpreter를 변경하고 쿼리문을 작성할 수 있다. 근데 난 왜 안되지?

In [18]:
%%sql
-- Query all data
select * from swimmers

In [20]:
# Query id and age for swimmers with age =22 via Dataframe API
swimmers.select("id","age").filter("age=22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [21]:
# Query id and age for swimmers with age = 22 in SQL 
spark.sql("select id, age from swimmers where age =22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [22]:
%%sql
#--Query id and age for swimmers with age =22
select id,age from swimmers where age = 22 

In [23]:
#Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



### Querying with the DataFrame API
 - With DataFrames, you can start writing your queries using the DataFrame API

In [24]:
#show the values
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [25]:
# Using Databricks 'display' command to view the data easier
display(swimmers)

NameError: name 'display' is not defined